In [ ]:
!pip install rasterio shapely opencv-python
!pip install fiona geopandas
!pip install pystac_client planetary_computer

In [ ]:
!pip install xarray rioxarray


In [ ]:
from pystac_client import Client

# Connect to Planetary Computer STAC
stac_url = "https://planetarycomputer.microsoft.com/api/stac/v1"
stac_client = Client.open(stac_url)

# Search for Sentinel-2 images
search = stac_client.search(
    collections=["sentinel-2-l2a"],
    bbox=[-122.52, 37.70, -122.35, 37.85],  # Bounding box (longitude, latitude)
    datetime="2015-01-01/2020-12-31",  # Time range
)

# Get matching items
items = list(search.get_items())
print(f"Found {len(items)} items")
# Print properties of first item
if items:
    print(items[0].properties.keys())


/usr/local/lib/python3.11/dist-packages/pystac_client/item_search.py:881: FutureWarning: get_items() is deprecated, use items() instead
  warnings.warn(


Found 307 items
dict_keys(['datetime', 'platform', 'instruments', 's2:mgrs_tile', 'constellation', 's2:granule_id', 'eo:cloud_cover', 's2:datatake_id', 's2:product_uri', 's2:datastrip_id', 's2:product_type', 'sat:orbit_state', 's2:datatake_type', 's2:generation_time', 'sat:relative_orbit', 's2:water_percentage', 's2:mean_solar_zenith', 's2:mean_solar_azimuth', 's2:processing_baseline', 's2:snow_ice_percentage', 's2:vegetation_percentage', 's2:thin_cirrus_percentage', 's2:cloud_shadow_percentage', 's2:nodata_pixel_percentage', 's2:unclassified_percentage', 's2:dark_features_percentage', 's2:not_vegetated_percentage', 's2:degraded_msi_data_percentage', 's2:high_proba_clouds_percentage', 's2:reflectance_conversion_factor', 's2:medium_proba_clouds_percentage', 's2:saturated_defective_pixel_percentage', 'proj:code'])


In [ ]:
import planetary_computer as pc
pc.settings.set_subscription_key(123456)

In [ ]:

import sys
sys.path.append("..")

import numpy as np
import matplotlib.pyplot as plt

import imageio
import shapely
import rasterio
from rasterio.mask import mask as rio_mask
from shapely import geometry
import cv2

from utils import tcm_algorithms as tcm
from utils import utils
from data import data_interface as DataInterface

In [ ]:

RASTERIO_BEST_PRACTICES = dict(
    CURL_CA_BUNDLE='/etc/ssl/certs/ca-certificates.crt',
    GDAL_DISABLE_READDIR_ON_OPEN='EMPTY_DIR',
    AWS_NO_SIGN_REQUEST='YES',
    GDAL_MAX_RAW_BLOCK_CACHE_SIZE='200000000',
    GDAL_SWATH_SIZE='200000000',
    VSI_CURL_CACHE_SIZE='200000000'
)

In [ ]:
def show_images(images, titles=None):
    #Function to plot images into a figure
    num_images = len(images)
    if titles is not None:
        assert len(titles) == num_images

    fig, axs = plt.subplots(1, num_images, figsize=(num_images*4, 4))
    axs = axs.flatten()
    for i in range(num_images):

        axs[i].imshow(images[i])
        if titles is not None:
            axs[i].set_title(titles[i])
        axs[i].axis("off")
        axs[i].get_xaxis().set_visible(False)
        axs[i].get_yaxis().set_visible(False)

    plt.show()
    plt.close()

In [ ]:
def running_mean(x, N):

    cumsum = np.cumsum(np.insert(x, 0, 0))
    return (cumsum[N:] - cumsum[:-N]) / float(N)

def largest_smaller(X, k):
    #Function to fond the index of the largest value in X smaller than k
    right_idx = X.searchsorted(k,'right')-1
    return right_idx

In [ ]:
import fiona


def get_all_geoms_from_file(fn):
    geoms = []
    with fiona.open(fn, driver='GeoJSON') as f: # Explicitly specify the driver
        for row in f:
            geom = row["geometry"]
            geoms.append(geom)
    return geoms

In [ ]:
# Add your planetary computer subscription key here

PC_SUBSCRIPTION_KEY = "123456"

In [ ]:
import os
import fiona

# Methods for getting solar farm geoms
def get_solar_farm_geoms(base_dir="./data/", polygons_fn="solar_farms_India_merged_4326.geojson"):
    # Correct the file path by directly joining the base directory and the file name.
    file_path = os.path.join(base_dir, polygons_fn)
    return get_all_geoms_from_file(file_path)

def get_all_geoms_from_file(fn):
    geoms = []
    with fiona.open(fn, driver='GeoJSON') as f:  # Explicitly specify the driver
        for row in f:
            geom = row["geometry"]
            geoms.append(geom)
    return geoms

In [ ]:
def get_data_stack_from_geom(self, geom_idx, buffer):
    crss = set()
    for item in items:
        # Try different possible property keys or use a default
        epsg_code = item.properties.get("proj:epsg")  # This will return None if the key is not found
        if epsg_code:
            crss.add(epsg_code)
        else:
            print(f"Warning: 'proj:epsg' not found in item properties. Available keys: {item.properties.keys()}")


In [ ]:
import geopandas as gpd

# Load the GeoJSON file
gdf = gpd.read_file('/content/solar_farms_india_2021_merged.geojson')


len(geoms)
geom_id = 80
buffer = 10

In [ ]:
geoms = utils.get_solar_farm_geoms(base_dir="/content/",
                                  polygons_fn="solar_farms_india_2021_merged.geojson")


In [ ]:
# PC dataloader is instantiated
dataloader = DataInterface.PlanetaryComputerS2DataLoader(geoms, pc_subscription_key=PC_SUBSCRIPTION_KEY)

In [ ]:
# Add this at the beginning of the get_data_stack_from_geom method to debug
print("Available properties keys:", items[0].properties.keys())

Available properties keys: dict_keys(['datetime', 'platform', 'instruments', 's2:mgrs_tile', 'constellation', 's2:granule_id', 'eo:cloud_cover', 's2:datatake_id', 's2:product_uri', 's2:datastrip_id', 's2:product_type', 'sat:orbit_state', 's2:datatake_type', 's2:generation_time', 'sat:relative_orbit', 's2:water_percentage', 's2:mean_solar_zenith', 's2:mean_solar_azimuth', 's2:processing_baseline', 's2:snow_ice_percentage', 's2:vegetation_percentage', 's2:thin_cirrus_percentage', 's2:cloud_shadow_percentage', 's2:nodata_pixel_percentage', 's2:unclassified_percentage', 's2:dark_features_percentage', 's2:not_vegetated_percentage', 's2:degraded_msi_data_percentage', 's2:high_proba_clouds_percentage', 's2:reflectance_conversion_factor', 's2:medium_proba_clouds_percentage', 's2:saturated_defective_pixel_percentage', 'proj:code'])


In [ ]:
def get_data_stack_from_geom(self, geom_idx, buffer):
    crss = set()
    for item in items:
        # Use 'proj:code' instead of 'proj:epsg'
        epsg_code = item.properties.get("proj:code")
        if epsg_code:
            crss.add(epsg_code)
        else:
            print(f"Warning: 'proj:code' not found in item properties. Available keys: {item.properties.keys()}")

    # Continue with the rest of your function


In [ ]:
%%time
rgb_images, dates = dataloader.get_rgb_chips_from_geom(geom_id, buffer=buffer, show_outline=True)

/usr/local/lib/python3.11/dist-packages/pystac_client/item_search.py:881: FutureWarning: get_items() is deprecated, use items() instead
  warnings.warn(


Returned 41 Items


KeyError: 'proj:epsg'

In [ ]:
print("{} matching image chips found".format(len(rgb_images)))

NameError: name 'rgb_images' is not defined

In [ ]:
%%time
data_images, masks, dates = dataloader.get_data_stack_from_geom(geom_id, buffer=buffer)

In [ ]:
%%time
divergence_values = tcm.calculate_change_values(data_images, masks, n_clusters=64, use_minibatch=True)

In [ ]:

for i in range(len(dates)):
    plt.figure(figsize=(4,3))
    plt.imshow(rgb_images[i])
    plt.axis("off")
    plt.title(f"{dates[i]} ({i}) -- Divergence: {divergence_values[i]:0.4f}")
    plt.show()
    plt.close()

In [ ]:

plt.figure()
plt.plot(divergence_values)
plt.hlines(y=np.median(divergence_values), xmin=0, xmax=len(divergence_values), color='k')
plt.xlabel("Scene Number")
plt.ylabel("KL Divergence")
plt.show()
plt.close()

In [ ]:
tcm_offset = 1
built_idx = largest_smaller(smoothed_divergence_values, np.median(divergence_values)) + tcm_offset

In [ ]:
print("The solar farm is estimated to be built by: "+ dates[built_idx])

In [ ]:
show_images(rgb_images[built_idx - 2: built_idx + 3 ], dates[built_idx - 2: built_idx + 3 ])

In [ ]:
!git clone https://github.com/EricSiq/solar-farms-mapping/tree/fbd4e0bc243ce1fa09bf851cd8d26d37835bf95b/data/landcover


In [ ]:
landcover_urls = [
        "../data/landcover/2015_landcover_india.tif",
        "../data/landcover/2016_landcover_india.tif",
        "../data/landcover/2017_landcover_india.tif",
        "../data/landcover/2018_landcover_india.tif",
        "../data/landcover/2019_landcover_india.tif"

]

In [ ]:
landcover_cl = [0, 20, 30, 40, 50, 60, 70, 80, 90, 100, 111, 112, 113, 114, 115, 116, 121, 122, 124, 125, 126, 200]
landcover_cl_text = ["Unknown", "Shrubs", "Herbaceous vegetation", "Cultivated and managed vegetation / agriculture",
                     "Urban / built up", "Bare / sparse vegetation", "Snow and ice", "Permanent water bodies",
                    "Herbaceous wetland", "Moss and lichen", "Closed forest, evergreen needle leaf", "Closed forest, evergreen broad leaf",
                    "Closed forest, deciduous needle leaf", "Closed forest, deciduous broad leaf", "Closed forest, mixed",
                    "Closed forest", "Open forest, evergreen needle leaf.", "Open forest, evergreen broad leaf.", "Open forest, deciduous needle leaf.",
                    "Open forest, deciduous broad leaf", "Open forest, mixed.", "Open forest", "Oceans, seas" ]
c2i = {cl:i for i,cl in enumerate(landcover_cl)}
colors = np.array([[40, 40, 40],[255, 187, 34],[255, 255, 76],[240, 150, 255],[250, 0, 0], [180, 180, 180], [240, 240, 240], [0, 50, 200], [0, 150, 160], [250, 230, 160], [88, 72, 31], [0, 153, 0], [112, 102, 62], [0, 204, 0],[78, 117, 31], [0, 120, 0], [102, 96, 0], [141, 180, 0], [141, 116, 0], [160, 220, 0], [146, 153, 0], [100, 140, 0], [0, 0, 128]])/255.


def vis_lc(r):
    r = np.squeeze(r)
    z = np.zeros((3,) + r.shape)
    r = np.array([(r==landcover_cl[i]) for i in range(len(landcover_cl))])
    s = r / r.sum(0)
    for c in range(len(landcover_cl)):
        for ch in range(3):
            z[ch] += colors[c,ch] * s[c]
    z = np.rollaxis(z, 0, 3)

    return z

In [ ]:
def get_mask_and_bounding_geoms(geom, buffer):
    footprint_shape = shapely.geometry.shape(geom).buffer(0.0)
    bounding_shape = footprint_shape.envelope.buffer(buffer).envelope
    mask_geom = shapely.geometry.mapping(bounding_shape - footprint_shape) # full bounding area - initial footprint
    bounding_geom = shapely.geometry.mapping(bounding_shape) # full bounding area
    return mask_geom, bounding_geom

def get_landcover_stack_from_geom(geom, buffer, urls):
    mask_geom, bounding_geom = get_mask_and_bounding_geoms(geom, buffer)
    images = []
    labels = []
    for url in urls:
        with rasterio.Env(**RASTERIO_BEST_PRACTICES):
            with rasterio.open(url) as f:
                mask_image, _ = rio_mask(f, [mask_geom], crop=True, invert=False, pad=False, all_touched=True)

                full_image, _ = rio_mask(f, [bounding_geom], crop=True, invert=False, pad=False, all_touched=True)
                landcover = vis_lc(full_image)
        images.append((landcover))
        labels.append((full_image))
    return images, labels

In [ ]:
landcover, landcover_classes = get_landcover_stack_from_geom(geoms[geom_id], buffer=0, urls=landcover_urls)
from datetime import datetime
from dateutil.relativedelta import relativedelta

dates_list = [datetime.strptime(date, '%m-%d-%Y').date() for date in dates]
before_built_date= dates_list[built_idx] - relativedelta(years=1)
print(before_built_date)

In [ ]:
year=str(before_built_date.year)
landcover_idx = [i for i, j in enumerate(landcover_urls) if year in j]

imagery_idx = [i for i, j in enumerate(dates) if year in j]

In [ ]:
f, axarr = plt.subplots(1,2)
axarr[0].imshow(landcover[landcover_idx[0]])
axarr[0].set_title(f"LULC before built")
axarr[0].axis("off")
axarr[1].imshow(rgb_images[imagery_idx[0]])
axarr[1].set_title(f"S2 Imagery for: " + dates[imagery_idx[0]])
plt.axis("off")
plt.show()

In [ ]:

lc, counts = np.unique(landcover_classes[landcover_idx[0]], return_counts=True)

In [ ]:
max_count = max(counts)
landcover_mode = lc[list(counts).index(max_count)]

In [ ]:
print("The most common LCLU class for where this solar farm was built was: ",landcover_cl_text[list(landcover_cl).index(landcover_mode)])